### import of useful libraries
math for ceil, networkx for network capabilities, pandas to read excel 

In [10]:
import networkx as nx
import math
import pandas as pd

### definition of target and machine type
the machine type is read from excel file. we also calculate how many machine of each type do we need max to reach the target. To help with the computational time, we start with the fewer choices configurations. So machines list is sorted according to how many machines of each type is needed to reach the target.

In [6]:
target_heating = 1500
target_cooling = 1500

df = pd.read_excel('/Users/fabien/Downloads/Tool_version_2.1.xlsx',sheet_name='Calcul',usecols='E:J',header=1)

machines = []
for i,row in df.iterrows():
    heating = row['Heating capacity'] if not math.isnan(row['Heating capacity']) else 0
    cooling = row['Cooling Capacity'] if not math.isnan(row['Cooling Capacity']) else 0
    
    if heating > 0:
        if cooling > 0:
            max_target = math.ceil(max(target_heating/heating,target_cooling/cooling))
        else:
            max_target = math.ceil(target_heating/heating)
    else:
        max_target = math.ceil(target_cooling/cooling)
    
    machine = {'type':row['Devices'],
               'price':row['Cost per unit'],
               'heating':row['Heating capacity'] if not math.isnan(row['Heating capacity']) else 0,
               'cooling':row['Cooling Capacity'] if not math.isnan(row['Cooling Capacity']) else 0,
               'max':max_target
              }
    machines.append(machine)
machines.sort(key=lambda a:a['max'],reverse=False)

### the optimization algorithm
This algorithm work layer by layer. Each machine type is a layer. Each time a new layer is added, the nodes of the new layer contains the current heating, cooling and price. Is we reach target already, we check the price. If the current solution do not beat the best price so far, we do not add the node as it is unsuitable. 

In [7]:
G = nx.DiGraph()
G.add_node(0)

# creation du graph
best_price_so_far = None

for i in range(0,len(machines)):
    name_i = machines[i]['type']
    price_i = machines[i]['price']
    max_i = machines[i]['max']
    heating_i = machines[i]['heating']
    cooling_i = machines[i]['cooling']
    print('machine type',name_i,'can be put only',max_i,'times to match target')
    
    if i == 0:
        for ri in range(0,max_i+1):
            G.add_node(name_i+'_'+str(ri),heating=ri*heating_i,cooling=ri*cooling_i,price=ri*price_i)
            G.add_edge(0, name_i+'_'+str(ri))
    else:
        node_list = [node for (node,degree) in G.degree() if degree==1]
        print('node_list len',len(node_list))
        for node in node_list:
            heating_prev = G.nodes[node]['heating']
            cooling_prev = G.nodes[node]['cooling']
            price_prev = G.nodes[node]['price']
                
            if heating_prev > target_heating and cooling_prev > target_cooling:
                if best_price_so_far is not None:
                    if price_prev < best_price_so_far:
                        best_price_so_far = price_prev
                else:
                    best_price_so_far = price_prev
                
                continue
            
            if best_price_so_far is not None:
                if price_prev > (best_price_so_far+0.2*best_price_so_far):
                    continue
            
            if heating_prev > target_heating:
                 if cooling_i == 0:
                    continue
            if cooling_prev > target_cooling:
                if heating_i == 0:
                    continue
            
            for ri in range(0,max_i+1): 
                if best_price_so_far is not None:
                    if (price_prev + ri*price_i) > (best_price_so_far+0.2*best_price_so_far):
                        continue
                G.add_node(node+'|'+name_i+'_'+str(ri),heating=heating_prev + ri*heating_i,cooling=cooling_prev + ri*cooling_i,price=price_prev + ri*price_i)
                G.add_edge(node, node+'|'+name_i+'_'+str(ri)) 


node_list = [(node,G.nodes[node]['price']) for (node,degree) in G.degree() if degree==1 and G.nodes[node]['heating']>=target_heating and G.nodes[node]['cooling']>=target_cooling]
print('candidate list',len(node_list))

machine type Groud source heat pump can be put only 1 times to match target
machine type Air-cooled heat pump can be put only 1 times to match target
node_list len 2
machine type Water cooled Chillers can be put only 1 times to match target
node_list len 4
machine type Cooling towers can be put only 1 times to match target
node_list len 8
machine type Ice storage can be put only 4 times to match target
node_list len 12
machine type AC can be put only 6 times to match target
node_list len 24
machine type Roof AC can be put only 6 times to match target
node_list len 42
machine type Furnace  can be put only 19 times to match target
node_list len 190
machine type Unit heaters can be put only 38 times to match target
node_list len 853
machine type Boilers can be put only 417 times to match target
node_list len 12672
machine type Radiant heaters can be put only 500 times to match target
node_list len 19902
machine type Burners can be put only 1000 times to match target
node_list len 243527
c

In [8]:
#top 10 solutions : 
node_list.sort(key=lambda a:a[1])
for i,(node,price) in enumerate(node_list[:10]):
    heating = G.nodes[node]['heating']
    cooling = G.nodes[node]['cooling']
    sources = [source for source in str(node).split('|') if source.split('_')[1] != '0']
    print('Solution','#'+str(i+1),'for HEATING',target_heating,'and COOLING',target_cooling)
    print('\tHeating Total',heating)
    print('\tCooling Total',cooling)
    print('\tPrice Total',price)
    print('\tDevices')
    for source in sources:
        device = source.split('_')[0]
        quantity = source.split('_')[1]
        print('\t',quantity,'x',device)
    print()

Solution #1 for HEATING 1500 and COOLING 1500
	Heating Total 1520.0
	Cooling Total 1600.0
	Price Total 183855.0
	Devices
	 4 x Ice storage
	 38 x Unit heaters

Solution #2 for HEATING 1500 and COOLING 1500
	Heating Total 1501.0
	Cooling Total 1600.0
	Price Total 188212.5
	Devices
	 4 x Ice storage
	 37 x Unit heaters
	 7 x Radiant heaters

Solution #3 for HEATING 1500 and COOLING 1500
	Heating Total 1504.0
	Cooling Total 1600.0
	Price Total 189052.5
	Devices
	 4 x Ice storage
	 37 x Unit heaters
	 8 x Radiant heaters

Solution #4 for HEATING 1500 and COOLING 1500
	Heating Total 1507.0
	Cooling Total 1600.0
	Price Total 189892.5
	Devices
	 4 x Ice storage
	 37 x Unit heaters
	 9 x Radiant heaters

Solution #5 for HEATING 1500 and COOLING 1500
	Heating Total 1510.0
	Cooling Total 1600.0
	Price Total 190732.5
	Devices
	 4 x Ice storage
	 37 x Unit heaters
	 10 x Radiant heaters

Solution #6 for HEATING 1500 and COOLING 1500
	Heating Total 1513.0
	Cooling Total 1600.0
	Price Total 191572.5